# Mean Reversion

## Internal Bar Strength

### Theory

**IBS** can in someway predict bullish or bearish movement of feature. It' pretty simple to calculate:
$$
\text{IBS}=\frac{\text{Close}-\text{Low}}{\text{High}-\text{Low}}
$$

$$
\text{IBS} < 0.6 \rightarrow \text{potential bearish sentiment}
$$
$$
\text{IBS} \geq 0.6 \rightarrow \text{potential bullish sentiment}
$$

Bollinger Band integrate historical data to estimate when a mean reversal is probable. Bollinger is using 25-day moving average of high prices and its deviation from the scaled range of high and low moving average prices:

$$
\text{Band} = \text{MA}_{25}(\text{High})-(2.25\cdot\text{HL}_\text{avg})
$$
where
$$
\text{HL}_\text{avg}=\text{MA}_{25}(\text{High})-\text{MA}_{25}(\text{Low})
$$

### Fetch data

In [160]:
import yfinance as yf

ticker = "^GSPC"
ticker = "ORCL"

data = yf.download(ticker, period="1y", interval="1d", group_by="column")
data.columns = data.columns.droplevel(1)  # drop second level of MultiIndex

/var/folders/g3/sxyl_n7x6d51bbb320nhxx5c0000gn/T/ipykernel_1059/2019963194.py:6: FutureWarning: YF.download() has changed argument auto_adjust default to True
  data = yf.download(ticker, period="1y", interval="1d", group_by="column")
[*********************100%***********************]  1 of 1 completed


In [161]:
print(data)

Price            Close        High         Low        Open    Volume
Date                                                                
2024-08-16  136.046448  136.392829  134.977630  135.373482   4893800
2024-08-19  136.491791  136.521479  135.046918  136.105830   3641600
2024-08-20  137.728836  138.342411  136.392825  136.392825   3738300
2024-08-21  138.748169  138.847139  137.115261  137.362673   3776800
2024-08-22  136.650116  139.619050  136.224561  139.242980   5162200
...                ...         ...         ...         ...       ...
2025-08-11  252.679993  254.690002  245.559998  248.399994   9012100
2025-08-12  253.860001  257.869995  251.080002  252.580002   9998400
2025-08-13  244.179993  257.670013  242.639999  257.170013  14182100
2025-08-14  244.960007  248.919998  242.570007  244.949997  10285600
2025-08-15  248.279999  250.630005  242.929993  246.919998  11537800

[250 rows x 5 columns]


### Prepare data to Bollinger Band

In [162]:
data['IBS'] = (data['Close'] - data['Low']) / (data['High'] - data['Low'])
data['HL_avg'] = data['High'].rolling(window=25).mean() - data['Low'].rolling(window=25).mean()
data['Band'] = data['High'].rolling(window=25).mean() - (2.25 * data['HL_avg'])

In [163]:
print(data.tail())

Price            Close        High         Low        Open    Volume  \
Date                                                                   
2025-08-11  252.679993  254.690002  245.559998  248.399994   9012100   
2025-08-12  253.860001  257.869995  251.080002  252.580002   9998400   
2025-08-13  244.179993  257.670013  242.639999  257.170013  14182100   
2025-08-14  244.960007  248.919998  242.570007  244.949997  10285600   
2025-08-15  248.279999  250.630005  242.929993  246.919998  11537800   

Price            IBS    HL_avg        Band  
Date                                        
2025-08-11  0.779846  6.136518  233.179497  
2025-08-12  0.409426  6.078818  233.987000  
2025-08-13  0.102461  6.500001  233.905798  
2025-08-14  0.376378  6.491601  234.279099  
2025-08-15  0.694805  6.634401  234.622998  


### Backtesting strategy

**In this backtesting we are skipping taxes and brokerage commissions.**
1. For loop to simulate every day from 25th day to end of list.
2. If position on market doesn't exist + IBS is below 0.6 and closing price is below Band then buy.
3. If position on market exist + today closing price is higher than High price from yesterday then sell.

In [ ]:
is_position = False
assets_num = 0
wallet = 1000

backtesting_data = {'Entry': [], 'Exit': []}

for i in range(25, len(data)):
    if not is_position and data['IBS'].iloc[i] < 0.6 and data['Close'].iloc[i] < data['Band'].iloc[i]:
        is_position = True
        assets_num = wallet / data['Close'].iloc[i]
        wallet = 0
        backtesting_data['Entry'].append({
            'Date': data.index[i],
            'Price': data['Close'].iloc[i]
        })
    elif is_position and data['Close'].iloc[i] > data['High'].iloc[i - 1]:
        is_position = False
        wallet = assets_num * data['Close'].iloc[i]
        assets_num = 0
        backtesting_data['Exit'].append({
            'Date': data.index[i],
            'Price': data['Close'].iloc[i]
        })

In [165]:
print(wallet)

982.3585551692189


In [166]:
print(len(backtesting_data['Entry']), "entries")
print(len(backtesting_data['Exit']), "exits")

10 entries
10 exits


In [167]:
for entry, exit in zip(backtesting_data['Entry'], backtesting_data['Exit']):
    print(f"Entry: {entry}, Exit: {exit} - Error: Entry after Exit")


Entry: 2024-12-11 00:00:00, Exit: 2024-12-24 00:00:00 - Error: Entry after Exit
Entry: 2024-12-26 00:00:00, Exit: 2025-01-14 00:00:00 - Error: Entry after Exit
Entry: 2025-01-15 00:00:00, Exit: 2025-01-16 00:00:00 - Error: Entry after Exit
Entry: 2025-01-27 00:00:00, Exit: 2025-01-30 00:00:00 - Error: Entry after Exit
Entry: 2025-03-06 00:00:00, Exit: 2025-03-12 00:00:00 - Error: Entry after Exit
Entry: 2025-03-13 00:00:00, Exit: 2025-03-17 00:00:00 - Error: Entry after Exit
Entry: 2025-03-18 00:00:00, Exit: 2025-03-19 00:00:00 - Error: Entry after Exit
Entry: 2025-03-27 00:00:00, Exit: 2025-04-01 00:00:00 - Error: Entry after Exit
Entry: 2025-04-03 00:00:00, Exit: 2025-04-09 00:00:00 - Error: Entry after Exit
Entry: 2025-04-16 00:00:00, Exit: 2025-04-22 00:00:00 - Error: Entry after Exit
